In [ ]:
import pyemma
pyemma.__version__
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pyemma.msm as msm
import pyemma.plots as mplt
import mdtraj
import sys
import glob
import csv
import os 

In [ ]:
import MDAnalysis as mda
import MDAnalysis.transformations
import itertools
from tqdm import tqdm_notebook as tqdm
import glob
import pyemma.coordinates as coor

In [ ]:
from MDAnalysis import Universe

In [ ]:
%pwd

In [ ]:
current_dir=os.getcwd()
print(current_dir)

In [ ]:
all_reader=[]
for index,name in enumerate(sorted(glob.glob("../gerbil_main/candi*/*/RC.txt"))):
        all_reader.append(np.loadtxt(name))
all_Y=all_reader
print(np.shape(all_Y))

In [ ]:
n_clusters =100
clustering = coor.cluster_kmeans(all_Y,k=n_clusters, stride=100, max_iter=5000)
dtrajs = list(np.reshape(clustering.dtrajs,np.shape(all_Y)[0:2]))
np.shape(dtrajs)

In [ ]:
plt.figure(figsize=(12,8))
mplt.plot_free_energy(np.vstack(all_Y)[:,0], np.vstack(all_Y)[:,1], levels=10, cmap="viridis")
cc_x = clustering.clustercenters[:,0]
cc_y = clustering.clustercenters[:,1]
plt.plot(cc_x,cc_y, linewidth=0, marker='o', markersize=3, color='black')
plt.gca().set_aspect("equal")

In [ ]:
its = msm.timescales_msm(dtrajs, lags=500, nits=10, show_progress=True)

In [ ]:
plt.figure(figsize=(8,5))
matplotlib.rcParams.update({'font.size': 14})
mplt.plot_implied_timescales(its, show_mean=False, ylog=True, dt=1, units='ps', linewidth=2)
plt.xlim(0, 100); plt.ylim(1,100000);

In [ ]:
msm_lag =30
M = msm.estimate_markov_model(dtrajs, msm_lag)
print('fraction of states used = ', M.active_state_fraction)
print('fraction of counts used = ', M.active_count_fraction)

In [ ]:
M = msm.bayesian_markov_model(dtrajs, msm_lag)

In [ ]:
xall = np.vstack(all_Y)[:,0]
yall = np.vstack(all_Y)[:,1]
W = np.concatenate(M.trajectory_weights())
np.savetxt("weight.xvg",W)

In [ ]:
open_RC=np.loadtxt("OP_RC.txt")
closed_RC=np.loadtxt("CL_RC.txt")

In [ ]:
all_reader=[]
for index,name in enumerate(sorted(glob.glob("./candi_*/RC.txt"))):
        all_reader.extend(np.loadtxt(name))
np.savetxt("all_RC.txt",np.array(all_reader))

In [ ]:
!paste all_RC.txt weight.xvg > string_input.xvg

In [ ]:
matplotlib.rcParams.update({'font.size': 8})
plt.figure(figsize=(8,5))
mplt.plot_free_energy(xall, yall, weights=W, levels=20,cmap="Blues_r",nbins=20,ax=plt.gca(),cbar_label='')
plt.scatter(open_RC[0],open_RC[1],s=25,c="cyan",edgecolor="black",zorder=999)
plt.text(open_RC[0],open_RC[1]+8, "4AKE\n(OP)",c="white")
plt.scatter(closed_RC[0],closed_RC[1],s=25,c="cyan",edgecolor="black",zorder=999)
plt.text(closed_RC[0]-10,closed_RC[1]-25, "1AKE\n(Cl)",c="black")
plt.xlabel("PC1 [Å]")
plt.ylabel("PC2 [Å]")
plt.xlim(-80,100)
plt.ylim(-100,100)
plt.tight_layout()
plt.savefig("Only" + "_FEL.pdf")

In [ ]:
all_reader=[]
for index,name in enumerate(sorted(glob.glob("./candi_*/RC.txt"))):
        all_reader.extend(np.loadtxt(name))
np.savetxt("all_RC.txt",np.array(all_reader))

In [ ]:
def plot_nice_pmf(xall,yall,weights,n_bins=25,ax=None):
    H = np.histogram2d(xall,yall, bins=n_bins, weights=weights)
    datanumber = np.sum(H[0])
    with np.errstate(divide='ignore'):
        logedH = -np.log(H[0].T/datanumber)
    minimalenagy = np.min(logedH)
    Z=logedH-minimalenagy
    x=[]
    for i in range(len(H[1])-1):
        x.append((H[1][i]+H[1][i+1])/2)
    y=[]
    for i in range(len(H[2])-1):
        y.append((H[2][i]+H[2][i+1])/2)
    X, Y = np.meshgrid(x, y)
    plt.contourf(X, Y, Z,levels=20,cmap="Blues_r")
    plt.colorbar()
    plt.contour(X, Y, Z,levels=20,colors="white",linewidths=0.1)

In [ ]:
timeseries=np.loadtxt("string_RC.xvg")
skip=2
matplotlib.rcParams.update({'font.size': 8})
plt.figure(figsize=(83/25.4,2.5))
plot_nice_pmf(xall,yall,W,ax=plt.gca())
plt.plot(timeseries[::skip,0], timeseries[::skip,1],color="yellow",marker='.', linewidth=1, markersize=3)
plt.scatter(open_RC[0],open_RC[1],s=25,c="#F8B62D",edgecolor="black",zorder=999)
plt.text(open_RC[0],open_RC[1]+8, "4AKE\n(OP)",c="white")
plt.scatter(closed_RC[0],closed_RC[1],s=25,c="#F8B62D",edgecolor="black",zorder=999)
plt.text(closed_RC[0]-20,closed_RC[1]+20, "1AKE(Cl)",c="black")
plt.xlabel("PC1 [Å]")
plt.ylabel("PC2 [Å]")
plt.xlim(-80,100)
plt.ylim(-100,100)
plt.tight_layout()
plt.savefig("GERBIL_reMSM_FEL.pdf")